# Считаем ембединги

Настройка colab

In [10]:
!nvidia-smi

Mon Mar 29 12:07:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 2.0MB 14.2MB/s 
     |████████████████████████████████| 1.2MB 51.8MB/s 
     |████████████████████████████████| 890kB 65.0MB/s 
     |████████████████████████████████| 3.2MB 68.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=f1dd9d3e474a0ebc211c891a383ba4affcf56aacc0ef3e1326e936b60df6bee6
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bb59a63418a4532b6c0df05d8aea6abad3ae7dd80d3a5ade7f4c43ac64d042e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Библиотеки

In [12]:
import os
#from google.colab import drive
from tqdm import tqdm
import pickle

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, util

My telegram bot

In [13]:
import json 
bot_api_file = "/content/drive/MyDrive/Colab Notebooks/opinion_analyzer/tg_bot.json"
tg_dict = json.load(open(bot_api_file, "r"))

In [14]:
import requests
from datetime import datetime 

TELEGRAM_TOKEN = tg_dict['TELEGRAM_TOKEN']
TELEGRAM_CHAT_ID = tg_dict['TELEGRAM_CHAT_ID']

def send_bot_message(msg, silent = True, has_output = True, 
                     TELEGRAM_TOKEN = TELEGRAM_TOKEN, TELEGRAM_CHAT_ID = TELEGRAM_CHAT_ID):   
    t = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    formatted_msg = "{message}     \n<b>{time}</b>".format(message = msg, time = t)
    msg_data = {
        'chat_id': TELEGRAM_CHAT_ID,
        'text': formatted_msg,
        'parse_mode': 'HTML',
        "disable_notification": silent
    }
    r = requests.post("https://api.telegram.org/bot{token}/sendMessage".format(token=TELEGRAM_TOKEN),
                         data=msg_data).content
    if has_output:
        print(msg)

## Загрузка файлов

Проверям папку

In [15]:
comments_dir = "/content/drive/MyDrive/comptech_opinion_analizer/data/data_reddit_clean"
os.listdir(comments_dir)[:5]

['JoeBiden_comments_2020_clean.csv', 'Trump_comments_2020_clean.csv']

Загружаем файл

In [16]:
%%time
csv_file = "Trump_comments_2020_clean.csv"
name_csv = csv_file.split("_")[0]
df_path = os.path.join(comments_dir, csv_file)

current_pd = pd.read_csv(df_path, index_col=0)

CPU times: user 1.87 s, sys: 236 ms, total: 2.11 s
Wall time: 2.2 s


In [17]:
current_pd.head()

,author,body,created_utc,link_id,parent_id,score
0,khalabrakis,"Fighters... That means warrior, like people wh...",1577836803,t3_ei6h2x,t3_ei6h2x,1
1,Whopper_Jr,it is plausible that Republicans will never wi...,1577836806,t3_ei4ag7,t3_ei4ag7,1
2,Sustainable_Saltmine,we must send bartenders back to bartending and...,1577836807,t3_eiay7b,t3_eiay7b,1
3,TheC0zmo,Bullshit. you are blaming a fiscally conservat...,1577836814,t3_ei40gl,t1_fco5fri,1
4,brotherjustincrowe,"Military R&amp;D and intel is not ""no reason.""...",1577836818,t3_ei51lq,t1_fcodqek,1


ГОТОВИМ ПАПКУ ДЛЯ ВЫВОДА

In [19]:
results_dir = "/content/drive/MyDrive/Colab Notebooks/opinion_analyzer/embeddings"
os.makedirs(results_dir, exist_ok = True)

# Векторизация комментариев

загружаем предобученную модель под сберта

In [20]:
#model = SentenceTransformer('bert-base-nli-mean-tokens') #эта быстрее
model = SentenceTransformer('bert-large-nli-mean-tokens') #эта точнее
model.get_max_seq_length()

128

Задаем бэтчи векторов по которым стоит пройтись

In [21]:
thresh_iter = 100000
threshold_list = np.arange(0, int(np.ceil(len(current_pd)/thresh_iter)*thresh_iter) + 1, thresh_iter)
iter_embedding_sets = list(zip(threshold_list, threshold_list[1:]))

In [22]:
for s, t in tqdm(iter_embedding_sets):
    job_pd = current_pd[s:t]
    body_vecs = []
    for index, row in job_pd.iterrows():
        body_vecs.append((row.link_id, row.body))
    
    link_ids, bodies = zip(*body_vecs)
    embeddings = model.encode(bodies, show_progress_bar = True, convert_to_numpy = True, device='cuda')

    # embedding_dict = {l:b for l,b in zip(link_ids, embeddings)}
    embeddings_list = list(embeddings)
    job_pd["embedding"] = embeddings_list

    # OUTPUT THE PD
    out_path = os.path.join(results_dir, f"{name_csv}_{s}-{t}.pickle")
    pickle.dump(job_pd, open(out_path, "wb"))
    msg = f"Job done :: {name_csv} :: between {s} - {t}"
    send_bot_message(msg)

    # TRASH OUT 
    del job_pd
    embeddings_list.clear() 
    body_vecs.clear()

  0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
 14%|█▍        | 1/7 [08:19<49:59, 499.98s/it]

Job done :: Trump :: between 0 - 100000


 29%|██▊       | 2/7 [17:27<42:50, 514.14s/it]

Job done :: Trump :: between 100000 - 200000


...

Job done :: Trump :: between 400000 - 500000


 86%|████████▌ | 6/7 [51:25<08:26, 506.36s/it]

Job done :: Trump :: between 500000 - 600000


100%|██████████| 7/7 [53:00<00:00, 454.39s/it]

Job done :: Trump :: between 600000 - 700000
